# VGG

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms
import torchvision
from train_func import *

In [2]:
class VGG(nn.Module):
    def __init__(self,in_channel,vgg_arch):
        super(VGG,self).__init__()
        self.image  = self.vgg_block(in_channel,vgg_arch)
        self.linear = self.linear_(vgg_arch[-1][-1])
    def forward(self,x):
        x=self.image(x)
        x=self.linear(x)
        return x


    def vgg_block(self,in_channel,vgg_arch):
        conv_block=[]
        for nums,out_channel in vgg_arch:
            conv_block.append(self.block_(nums,in_channel,out_channel))
            in_channel=out_channel
        conv_block.append(nn.Flatten())
        return nn.Sequential(*conv_block)
    
    def block_(self,nums,in_channel,out_channel):
        layers=[]
        # add convolution layers (didnt change shape)
        for _ in range(nums):
            layers.append(nn.Conv2d(in_channels=in_channel,out_channels=out_channel,kernel_size=3,padding=1))
            layers.append(nn.ReLU())
            in_channel=out_channel
        # reduce shape by half using maxpool
        layers.append(nn.MaxPool2d(kernel_size=2,stride=2))
        return nn.Sequential(*layers)
    
    def linear_(self,out_channels):
        layer=nn.Sequential(
            nn.Linear(out_channels * 7 * 7, 4096), nn.ReLU(), nn.Dropout(0.5),
            nn.Linear(4096, 4096), nn.ReLU(), nn.Dropout(0.5),
            nn.Linear(4096, 10)
        )
        return layer

            

In [3]:
# VGG-11
vgg_arch=((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))
model=VGG(3,vgg_arch)
model

VGG(
  (image): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (1): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (2): Sequential(
      (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
      (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU()
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (3): Sequential(
      (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
      (2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU()
      (4): MaxPool2d(kernel_size=2

In [4]:
train_dl,test_dl = get_CIFAR10(size=224,batch_size=256)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
model=VGG(3,vgg_arch).cuda()
optimizer=torch.optim.AdamW(model.parameters(),lr=0.0005) 
train_loop(model,optimizer,train_dl,test_dl,10)